In [ ]:
"""
페어 트레이딩 전략(Strategy)의 성과를 비교 분석하고 시각화

주요 기능 및 처리 로직:
    1. 경로 처리 (Path Handling):
        - GATEV, VIDYAMURTHY: 'BASE_YEAR' 하위 폴더를 사용하지 않습니다.
        - ISEPT 전략: 'BASE_YEAR' 하위 폴더를 사용합니다.
        - ISEPT 전략(2_GATEV, 2_VIDYAMURTHY) 예외 처리:
            * BEST_PAIR 경로: 'OURS' 경로를 사용 (BASE_YEAR 포함)
            * ALL_RESULTS 경로: '2_GATEV' 경로를 사용 (BASE_YEAR 포함)
            
    2. 데이터 집계 (Data Aggregation):
        - 월별 상위 TOP_K개 페어만 선정하여 평균 성과를 계산합니다.
        - NaN 값이 하나라도 포함된 페어는 제외하고 남은 페어로만 평균을 산출합니다.
        
    3. 시각화 (Visualization):
        - 범례(Legend) 정렬: 기본적으로 평균값이 높은 순서대로 정렬합니다.
        - 예외 정렬: Count, Volatility, Maximum Drawdown은 낮은 값이 상위에 오도록 정렬합니다.
        
    4. 디버깅 (Debugging):
        - 각 전략별 파일 경로와 읽어들인 기간(Period) 개수를 로그로 출력합니다.
"""

import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

# ──────────────────────────────────────────────────────────────────────────────
# 1. 설정 파라미터 및 상수 정의 (Configuration & Constants)
# ──────────────────────────────────────────────────────────────────────────────

# 분석 설정
TOP_K        = 20         # 상위 몇 개 페어를 사용하여 평균을 낼지 설정
start_period = "2004-01"  # 분석 시작 월
end_period   = "2024-06"  # 분석 종료 월
BASE_YEAR    = "1991"     # 데이터 베이스 연도 (폴더 구조용)

# 플롯용 색상 매핑 (TRADING_TYPE_LIST에 맞춰 추가/수정)
COLOR_MAP = {
    "GATEV": "C0",
    "VIDYAMURTHY": "C1",
    "2_GATEV": "C2",
    "2_VIDYAMURTHY": "C4",
}

# 처리할 전략 리스트 (분석 대상 TRADING_TYPE 나열)
TRADING_TYPE_LIST = [
    "GATEV",
    "VIDYAMURTHY",
    "2_GATEV",
    "2_VIDYAMURTHY"
]

# BEST_PAIR 경로 매핑 (TRADING_TYPE과 실제 디렉토리명이 다를 경우 사용)
BEST_PAIR_PATH_MAPPING = {
    # 2_GATEV, 2_VIDYAMURTHY는 BEST_PAIR 경로를 'OURS' 폴더에서 참조
    "2_GATEV": "OURS",
    "2_VIDYAMURTHY": "OURS"
}

# 경로 템플릿 정의 (BASE_DIR는 외부에서 주입된다고 가정)
# Case 1: BASE_YEAR 폴더가 없는 구조
BEST_PAIR_ROOT_NOYEAR     = os.path.join(BASE_DIR, "BEST_PAIR",   "TRAIN_ONE", "{TRADING_TYPE}")
ALL_RESULTS_ROOT_NOYEAR   = os.path.join(BASE_DIR, "PAIR_RESULT", "TRAIN_ONE", "{TRADING_TYPE}")

# Case 2: BASE_YEAR 폴더가 있는 구조
BEST_PAIR_ROOT_TEMPLATE   = os.path.join(BASE_DIR, "BEST_PAIR",   "TRAIN_ONE", "{TRADING_TYPE}", "{BASE_YEAR}")
ALL_RESULTS_ROOT_TEMPLATE = os.path.join(BASE_DIR, "PAIR_RESULT", "TRAIN_ONE", "{TRADING_TYPE}", "{BASE_YEAR}")

# 결과 출력 경로
OUTPUT_DIR_ROOT = os.path.join(BASE_DIR, "AVG_RESULT", f"TOTAL_TOP_{TOP_K}_ONLY")

# 집계할 메트릭 컬럼 (ALL_RESULTS CSV 헤더와 일치해야 함)
METRICS = [
    "Count",
    "ROI",
    "Sharpe Ratio",
    "Sortino Ratio",
    "Maximum Drawdown",
    "Calmar Ratio",
    "Volatility",
    "Hit Ratio",
]

# ──────────────────────────────────────────────────────────────────────────────
# 2. 유틸리티 함수 (Utility Functions)
# ──────────────────────────────────────────────────────────────────────────────

def parse_period(period_str: str):
    """
    'YYYY-MM' 형식의 문자열을 파싱하여 해당 월의 시작일과 종료일을 반환합니다.

    Args:
        period_str (str): 'YYYY-MM' 형식의 날짜 문자열

    Returns:
        tuple: (datetime, datetime) 형태의 (기간 시작일, 기간 종료일)
    """
    period_start = datetime.strptime(period_str + "-01", "%Y-%m-%d")
    next_month = period_start + relativedelta(months=1)
    return period_start, next_month - timedelta(days=1)

# ──────────────────────────────────────────────────────────────────────────────
# 3. 메인 로직 (Main Execution)
# ──────────────────────────────────────────────────────────────────────────────

def main():
    # 1) 분석 대상 기간 리스트 생성
    periods = pd.period_range(start=start_period, end=end_period, freq="M")
    period_list = [p.strftime("%Y-%m") for p in periods]

    all_df_avg = {}    # 전략별 월별 평균 DataFrame 저장용
    overall_avg = {}   # 전략별 전체 기간 평균 저장용

    # 2) 전략 유형별 루프 실행
    for TRADING_TYPE in TRADING_TYPE_LIST:
        print(f"\n=== Processing strategy: {TRADING_TYPE} ===")

        # ─── [2-1] BEST_PAIR 경로 설정 ───
        if TRADING_TYPE in {"GATEV", "VIDYAMURTHY"}:
            # Case A: BASE_YEAR 하위 폴더를 사용하지 않는 전략
            effective_best = BEST_PAIR_PATH_MAPPING.get(TRADING_TYPE, TRADING_TYPE)
            BEST_PAIR_BASE_DIR = BEST_PAIR_ROOT_NOYEAR.format(TRADING_TYPE=effective_best)
            
        elif TRADING_TYPE == "2_GATEV":
            # Case B: 2_GATEV (BEST_PAIR는 OURS 경로 사용, BASE_YEAR 포함)
            effective_best = BEST_PAIR_PATH_MAPPING.get(TRADING_TYPE, TRADING_TYPE)  # "OURS"
            BEST_PAIR_BASE_DIR = BEST_PAIR_ROOT_TEMPLATE.format(
                TRADING_TYPE=effective_best, BASE_YEAR=BASE_YEAR
            )
            
        else:
            # Case C: 기타 전략 (BASE_YEAR 하위 폴더 사용)
            effective_best = BEST_PAIR_PATH_MAPPING.get(TRADING_TYPE, TRADING_TYPE)
            BEST_PAIR_BASE_DIR = BEST_PAIR_ROOT_TEMPLATE.format(
                TRADING_TYPE=effective_best, BASE_YEAR=BASE_YEAR)

        # ─── [2-2] ALL_RESULTS 경로 설정 ───
        if TRADING_TYPE in {"GATEV", "VIDYAMURTHY"}:
            # Case A: BASE_YEAR 폴더 없음
            ALL_RESULTS_BASE_DIR = ALL_RESULTS_ROOT_NOYEAR.format(TRADING_TYPE=TRADING_TYPE)

        elif TRADING_TYPE == "2_GATEV":
            # Case B: 2_GATEV (ALL_RESULTS는 자기 자신 경로 사용, BASE_YEAR 포함)
            ALL_RESULTS_BASE_DIR = ALL_RESULTS_ROOT_TEMPLATE.format(
                TRADING_TYPE=TRADING_TYPE, BASE_YEAR=BASE_YEAR
            )

        else:
            # Case C: 기타 전략 (BASE_YEAR 폴더 있음)
            ALL_RESULTS_BASE_DIR = ALL_RESULTS_ROOT_TEMPLATE.format(
                TRADING_TYPE=TRADING_TYPE, BASE_YEAR=BASE_YEAR
            )

        print(f"[DEBUG] BEST_PAIR_BASE_DIR for {TRADING_TYPE}: {BEST_PAIR_BASE_DIR}")
        print(f"[DEBUG] ALL_RESULTS_BASE_DIR for {TRADING_TYPE}: {ALL_RESULTS_BASE_DIR}")

        # 월별 평균 저장용 리스트 초기화
        results_monthly = []
        all_monthly_rows = []

        # 출력 디렉토리 생성
        OUTPUT_DIR = os.path.join(OUTPUT_DIR_ROOT, TRADING_TYPE)
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        # 유효 데이터 기간 카운터
        count_periods_with_data = 0

        # ─── [2-3] 기간별 데이터 처리 루프 ───
        for period in tqdm(period_list, desc=f"{TRADING_TYPE} periods"):
            
            # (A) ALL_RESULTS 파일 읽기
            all_results_path = os.path.join(ALL_RESULTS_BASE_DIR, f"{period}_all_results.csv")
            if not os.path.isfile(all_results_path):
                print(f"[DEBUG] [{TRADING_TYPE}] ALL_RESULTS 없음: {all_results_path}")
                continue
            try:
                df_all = pd.read_csv(all_results_path, comment="#")
            except Exception:
                print(f"[WARNING] [{TRADING_TYPE}] ALL_RESULTS 읽기 실패: {all_results_path}")
                continue

            # (B) BEST_PAIR 목록 파일 읽기
            best_pair_path = os.path.join(BEST_PAIR_BASE_DIR, period, "best_pair_list.csv")
            if not os.path.isfile(best_pair_path):
                print(f"[DEBUG] [{TRADING_TYPE}] BEST_PAIR 없음: {best_pair_path}")
                continue
            try:
                df_pairs = pd.read_csv(best_pair_path, dtype=str)
            except Exception:
                print(f"[WARNING] [{TRADING_TYPE}] BEST_PAIR 읽기 실패: {best_pair_path}")
                continue

            # (C) Ticker 컬럼 파싱 ('pair' 컬럼 분리)
            if "pair" in df_pairs.columns:
                df_pairs[["ticker1", "ticker2"]] = (
                    df_pairs["pair"].astype(str).str.split("_", n=1, expand=True)
                )
            df_pairs = df_pairs.dropna(subset=["ticker1", "ticker2"]).reset_index(drop=True)
            if df_pairs.empty:
                print(f"[DEBUG] [{TRADING_TYPE}] df_pairs.empty for period {period}")
                continue

            # (D) ALL_RESULTS와 BEST_PAIR 매칭 준비
            cols_needed = {"ticker1", "ticker2"}.union(METRICS)
            df_all_pairs = df_all[[c for c in df_all.columns if c in cols_needed]].copy()
            
            # Ticker 순서가 바뀐 경우(양방향 매칭)를 대비해 뒤집은 데이터도 추가
            df_all_swapped = df_all_pairs.rename(columns={"ticker1": "ticker2", "ticker2": "ticker1"})
            df_all_pairs = pd.concat([df_all_pairs, df_all_swapped], ignore_index=True)

            # (E) BEST_PAIR 순서대로 상위 TOP_K개 페어 필터링
            selected_records = []
            for idx, row in df_pairs.iterrows():
                if len(selected_records) >= TOP_K:
                    break
                t1 = row["ticker1"]
                t2 = row["ticker2"]
                
                # 해당 페어의 성과 데이터 찾기
                df_match = df_all_pairs[
                    (df_all_pairs["ticker1"] == t1) & (df_all_pairs["ticker2"] == t2)
                ]
                if df_match.empty:
                    continue
                
                # 첫 번째 매칭 결과 사용
                rec = df_match.iloc[0]
                
                # 데이터 유효성 검사 (NaN 체크 및 형변환)
                metrics_numeric = {}
                skip_flag = False
                for m in METRICS:
                    if m not in rec or pd.isna(rec[m]):
                        skip_flag = True
                        break
                    try:
                        val = float(rec[m])
                    except Exception:
                        skip_flag = True
                        break
                    metrics_numeric[m] = val
                
                if skip_flag:
                    continue  # 결측치가 있으면 제외

                # 유효 레코드 저장
                record = {
                    "ticker1": t1,
                    "ticker2": t2,
                }
                for m in METRICS:
                    record[m] = metrics_numeric[m]
                selected_records.append(record)

            if not selected_records:
                print(f"[DEBUG] [{TRADING_TYPE}] period {period}: 유효 TOP_K 페어 없음")
                continue

            # DataFrame 변환
            df_topk = pd.DataFrame(selected_records)
            n_selected = len(df_topk)
            print(f"[DEBUG] [{TRADING_TYPE}] period {period}: 유효 페어 수 = {n_selected} (요청 TOP_K={TOP_K})")

            # (F) 선정된 페어 목록 CSV 저장
            sel_out_dir = os.path.join(OUTPUT_DIR, "selected_pairs")
            os.makedirs(sel_out_dir, exist_ok=True)
            sel_csv_path = os.path.join(sel_out_dir, f"{TRADING_TYPE}_{period}_selected.csv")
            
            df_save = df_topk.copy().reset_index(drop=True)
            df_save.insert(0, "Rank", df_save.index + 1)
            df_save["pair"] = df_save["ticker1"] + "_" + df_save["ticker2"]
            df_save.to_csv(sel_csv_path, index=False)

            # (G) 메트릭 평균 계산 및 저장
            df_metrics_num = df_topk[METRICS]
            metrics_mean = df_metrics_num.mean()
            
            row_out = {"Period": period}
            for m in METRICS:
                row_out[m] = metrics_mean.get(m, np.nan)
            
            results_monthly.append(row_out)
            all_monthly_rows.append(df_metrics_num)
            count_periods_with_data += 1

        print(f"[INFO] {TRADING_TYPE}: 월별 평균 계산에 사용된 기간 수 = {count_periods_with_data} / {len(period_list)}")

        # ─── [2-4] 전체 기간 평균 계산 ───
        if all_monthly_rows:
            combined = pd.concat(all_monthly_rows, ignore_index=True)
            overall_means = combined.mean()
            overall_avg[TRADING_TYPE] = {m: overall_means.get(m, np.nan) for m in METRICS}
        else:
            overall_avg[TRADING_TYPE] = {m: np.nan for m in METRICS}

        # ─── [2-5] 월별 평균 결과 CSV 저장 ───
        avg_out_path = os.path.join(OUTPUT_DIR, f"{TRADING_TYPE}_average_results_topK_only.csv")
        df_avg = pd.DataFrame(results_monthly)
        if not df_avg.empty and "Period" in df_avg.columns:
            df_avg = df_avg.sort_values("Period").reset_index(drop=True)
        else:
            df_avg = pd.DataFrame(columns=["Period"] + METRICS)
        df_avg.to_csv(avg_out_path, index=False, float_format="%.4f")
        all_df_avg[TRADING_TYPE] = df_avg

    # ──────────────────────────────────────────────────────────────────────────────
    # 4. 결과 시각화 (Visualization)
    # ──────────────────────────────────────────────────────────────────────────────
    
    # X축 라벨 설정
    periods_full = pd.period_range(start=start_period, end=end_period, freq="M")
    period_labels = [p.strftime("%Y-%m") for p in periods_full]
    x = np.arange(len(period_labels))
    tick_pos = [i for i, p in enumerate(period_labels) if p.endswith("-01")]
    tick_labels = [p[2:4] for p in period_labels if p.endswith("-01")]

    fig, axes = plt.subplots(2, 4, figsize=(16, 8), constrained_layout=True)
    axes = axes.flatten()

    # 오름차순(낮은 값이 좋은) 메트릭 정의
    ASC_ORDER_METRICS = {"Count", "Volatility", "Maximum Drawdown"}

    for idx, metric in enumerate(METRICS):
        ax = axes[idx]

        # 범례 정렬 함수: NaN 값을 처리하며 정렬
        def sort_key(tr):
            val = overall_avg.get(tr, {}).get(metric, np.nan)
            if np.isnan(val):
                # 오름차순 지표인 경우 NaN을 무한대로, 내림차순인 경우 -무한대로 보내 맨 뒤로 배치
                if metric in ASC_ORDER_METRICS:
                    return np.inf
                else:
                    return -np.inf
            return val

        # 기본은 내림차순(높은 값 우선), ASC_ORDER_METRICS는 오름차순(낮은 값 우선)
        reverse_flag = metric not in ASC_ORDER_METRICS
        sorted_types = sorted(TRADING_TYPE_LIST, key=sort_key, reverse=reverse_flag)

        for TR in sorted_types:
            avg_val = overall_avg[TR].get(metric, np.nan)
            df_avg = all_df_avg.get(TR, pd.DataFrame(columns=["Period"] + METRICS))
            
            # 시계열 데이터 매핑
            if not df_avg.empty and "Period" in df_avg.columns:
                df_idxed = df_avg.set_index("Period")
                y_vals = [df_idxed.get(metric, pd.Series()).get(p, np.nan) for p in period_labels]
            else:
                y_vals = [np.nan] * len(period_labels)

            color = COLOR_MAP.get(TR)
            label = f"{TR}: {avg_val:.4f}" if not np.isnan(avg_val) else TR
            
            ax.plot(x, y_vals, marker="o", markersize=1, linewidth=1, color=color, label=label)
            
            # 전체 평균선 표시
            if not np.isnan(avg_val):
                ax.axhline(avg_val, color=color, linestyle="--", linewidth=1)

        ax.set_title(metric, fontsize=10)
        ax.set_xticks(tick_pos)
        ax.set_xticklabels(tick_labels, fontsize=8)
        ax.yaxis.set_major_locator(MaxNLocator(nbins=6))
        ax.legend(fontsize=7, framealpha=0.5)

    fig.suptitle(
        f"Monthly Average Metrics Comparison (Top{TOP_K}) ({start_period} ~ {end_period})",
        fontsize=14,
    )
    plt.show()

    # ──────────────────────────────────────────────────────────────────────────────
    # 5. 전체 결과 요약 저장 (Summary Saving)
    # ──────────────────────────────────────────────────────────────────────────────
    df_overall = pd.DataFrame.from_dict(overall_avg, orient="index")
    df_overall.index.name = "Trading Type"
    df_overall = df_overall.reindex(columns=METRICS)
    
    summary_path = os.path.join(
        OUTPUT_DIR_ROOT,
        f"TOTAL_overall_average_topK_only_{start_period}_{end_period}_top{TOP_K}.csv",
    )
    os.makedirs(os.path.dirname(summary_path), exist_ok=True)
    df_overall.to_csv(summary_path, float_format="%.4f")

if __name__ == "__main__":
    main()